# MeloTTS-Japaneseを使ったテキスト→音声変換アプリ

## 準備以下のセルを実行して、必要なライブラリをインストールします。初回のみ実行してください。

In [ ]:
# 必要なライブラリのインストール (最初に一度だけ実行)!pip install git+https://github.com/myshell-ai/MeloTTS.git!python -m unidic download

## アプリの実装以下のセルを実行して、テキスト→音声変換アプリを起動します。

In [ ]:
# ライブラリのインポートimport osimport ioimport tempfilefrom IPython.display import Audio, display, HTMLimport ipywidgets as widgetsfrom melo.api import TTS# テキストを音声に変換する関数def text_to_speech(text, speed=1.0, device='cpu'):    """    日本語テキストを音声に変換する        Parameters:    -----------    text : str        変換する日本語テキスト    speed : float        発話速度 (1.0が標準)    device : str        使用するデバイス ('cpu' または 'cuda:0')            Returns:    --------    str        生成された音声ファイルのパス    """    # 一時ファイルを作成    temp_dir = tempfile.gettempdir()    output_path = os.path.join(temp_dir, 'output.wav')        # MeloTTSモデルの初期化    model = TTS(language='JP', device=device)    speaker_ids = model.hps.data.spk2id        # テキストから音声を生成    model.tts_to_file(text, speaker_ids['JP'], output_path, speed=speed)        return output_path# ファイルからテキストを読み込む関数def read_text_file(uploaded_file):    """    アップロードされたファイルからテキストを読み込む        Parameters:    -----------    uploaded_file : UploadedFile        アップロードされたテキストファイル            Returns:    --------    str        ファイルの内容    """    content = uploaded_file.read()    # エンコーディングを自動検出（日本語ファイルの場合、utf-8やshift-jisなど様々な可能性がある）    encodings = ['utf-8', 'shift-jis', 'euc-jp', 'iso-2022-jp']        for encoding in encodings:        try:            text = content.decode(encoding)            return text        except UnicodeDecodeError:            continue        # どのエンコーディングでも失敗した場合    raise ValueError("ファイルのエンコーディングを検出できませんでした。")# ファイルアップロード時の処理def on_upload_change(change):    """    ファイルがアップロードされたときの処理    """    if not change.new:        return        # 処理状況を表示    status_output.value = "ファイルを処理中..."        try:        # アップロードされたファイルからテキストを読み込む        uploaded_file = list(change.new.values())[0]        text = read_text_file(uploaded_file)                # テキストプレビューを表示        text_preview.value = text if len(text) <= 1000 else text[:1000] + "..."                # テキストを音声に変換        speed = float(speed_slider.value)        audio_path = text_to_speech(text, speed=speed)                # 音声を表示        audio_output.clear_output()        with audio_output:            display(Audio(audio_path))                # ダウンロードリンクを作成        with open(audio_path, 'rb') as f:            audio_data = f.read()                download_link.value = create_download_link(audio_data, 'output.wav', '音声ファイルをダウンロード')                status_output.value = "処理完了！"        except Exception as e:        status_output.value = f"エラーが発生しました: {str(e)}"# ダウンロードリンクを作成する関数def create_download_link(audio_data, filename, text):    """    音声データをダウンロードするためのHTMLリンクを作成        Parameters:    -----------    audio_data : bytes        ダウンロードするオーディオデータ    filename : str        ダウンロード時のファイル名    text : str        リンクに表示するテキスト            Returns:    --------    str        HTMLリンク    """    b64 = io.BytesIO(audio_data)    payload = b64.getvalue()    import base64    b64_str = base64.b64encode(payload).decode()    return f'<a href="data:audio/wav;base64,{b64_str}" download="{filename}">{text}</a>'# 速度変更時の処理def on_speed_change(change):    """    速度スライダーが変更されたときの処理    """    speed_value.value = f"発話速度: {change.new}x"# UIの作成upload_button = widgets.FileUpload(    accept='.txt',    multiple=False,    description='テキストファイルを選択')speed_slider = widgets.FloatSlider(    value=1.0,    min=0.5,    max=2.0,    step=0.1,    description='速度:',    continuous_update=False)speed_value = widgets.HTML(value="発話速度: 1.0x")status_output = widgets.HTML(value="ファイルをアップロードしてください。")text_preview = widgets.Textarea(    description='テキスト:',    placeholder='アップロードされたテキストがここに表示されます',    disabled=True,    layout=widgets.Layout(width='100%', height='200px'))audio_output = widgets.Output()download_link = widgets.HTML()# イベントハンドラの登録upload_button.observe(on_upload_change, names='value')speed_slider.observe(on_speed_change, names='value')# UIの表示display(widgets.HTML("<h1>MeloTTS-Japanese テキスト→音声変換アプリ</h1>"))display(widgets.HTML("<p>日本語のテキストファイルをアップロードして、自然な音声に変換します。</p>"))display(widgets.HBox([upload_button, widgets.VBox([speed_slider, speed_value])]))display(status_output)display(text_preview)display(widgets.HTML("<h3>生成された音声:</h3>"))display(audio_output)display(download_link)